TODO
1. extract features by googlenet
2. compute similarity
3. test on Place dataset

In [1]:
# load places205 dataset
# import hub
# ds = hub.load("hub://activeloop/places205")
# dataloader = ds.pytorch(num_workers = 2, shuffle = False, batch_size= 4)

# import numpy as np
# img = ds.images[42].numpy()
# print(img.shape)

# import matplotlib.pyplot as plt
# plt.imshow(img)

In [2]:
import torch
import torch.nn as nn
from googlenet_places205 import GoogLeNetPlaces205

model = GoogLeNetPlaces205()
model.load_state_dict(torch.load("googlenet_places205.pth"))

# define hook
features = {}
def get_middle_output(name):
    def hook(model, input, output):
        features[name] = (output.detach())
    return hook

# model.inception_3b_5x5.register_forward_hook(get_middle_output('icp1'))
model.inception_4b_5x5.register_forward_hook(get_middle_output('icp1'))

In [12]:
import os
from PIL import Image
from torchvision import transforms

def preprocess_singe_img(filename):
    input_image = Image.open(filename)
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    input_tensor = transform(input_image)
    input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model
    return input_batch

def get_features(input):
    output = model(input)
    return features['icp1'].flatten()

def get_similarity(input1, input2):
    cos = nn.CosineSimilarity(dim=0, eps=1e-6)
    return cos(input1, input2)


def process_images_from_folder(folder):
    image_features = []
    for filename in os.listdir(folder):
        if filename is not None:
            input = preprocess_singe_img(folder + '/' + filename)
            image_features.append(get_features(input))
    return image_features

# filename = ["images/road.png", "images/road2.png", "images/rail.png", "images/rail2.png"]
# outputs = []
# for img in filename:
#     input = preprocess(img)
#     outputs.append(get_features(input))

In [20]:
import numpy as np

database = "images/database"
query = "images/query"

database_features = process_images_from_folder(database)
query_features = process_images_from_folder(query)

similarity_matrix = np.zeros((len(query), len(database_features))) # Q by D

for i in range(len(query_features)):
    for j in range(len(database_features)):
        similarity_matrix[i][j] = get_similarity(query_features[i], database_features[j])

# print(query_features[9])
# print(database_features[9])
print(similarity_matrix)

tensor([-1.3158, -2.0469, -2.1182,  ...,  3.9142,  3.5797,  3.1912])
tensor([-1.2194, -1.7579, -1.6108,  ...,  3.6001,  3.4171,  3.1232])
[[0.90076178 0.88163835 0.87797463 0.89589965 0.86852586 0.9068318
  0.90239567 0.89438421 0.87929589 0.89844006]
 [0.8977223  0.89657336 0.88530189 0.90753168 0.87957031 0.90492189
  0.90046072 0.89172792 0.8838402  0.90702587]
 [0.89282602 0.88042587 0.87665522 0.89696538 0.86917925 0.91075933
  0.91158795 0.89736629 0.88143718 0.90064716]
 [0.89534479 0.88059169 0.87750459 0.89731002 0.86872315 0.91105425
  0.91323161 0.90013766 0.88387418 0.90176362]
 [0.90983176 0.8957417  0.89488769 0.90974045 0.88164067 0.9240213
  0.92625242 0.91371489 0.89778656 0.91378891]
 [0.89913219 0.88031209 0.88222921 0.89834303 0.86958474 0.91152889
  0.91242987 0.89911371 0.88263708 0.89947456]
 [0.89584845 0.87162364 0.87684631 0.89176923 0.86093408 0.9050982
  0.90351647 0.891101   0.8749907  0.89198327]
 [0.89071667 0.86750412 0.87093419 0.88755512 0.85755229 0.9